In [1]:
import sys
sys.path.insert(
    0, "/Users/anis/test_labnbook/test_3_indicateurs/traces_contribution/sources")
sys.path.insert(
    0, "/Users/anis/test_labnbook/test_3_indicateurs/traces_collaboration_Anis_dispersion/sources")
sys.path.insert(
    0, "/Users/anis/test_labnbook/test_3_indicateurs/traces_collaboration_Anis_sans_ponderation/sources")
import numpy as np
import config
import mysql.connector as mariadb
import pandas as pd


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 100)

try:
    conn = mariadb.connect(user=config.USER, password=config.PASSWORD,
                           host=config.HOST, database=config.DATABASE)
except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")

# Récupération du curseur
cursor = conn.cursor()


In [2]:
# Importer les sorties des algo qui calculent nos indicateurs  
data = pd.read_csv(
    "/Users/anis/test_labnbook/test_3_indicateurs/traces_contribution/tmp/Contribution.csv")
data.rename(columns={'indice_collab': 'equilibre_contribution'}, inplace=True)

co_ecriture = pd.read_csv(
    "/Users/anis/test_labnbook/test_3_indicateurs/traces_collaboration_Anis/tmp/collab_Anis.csv")['indice_collab']

co_ecriture_dispersion = pd.read_csv(
    '/Users/anis/test_labnbook/test_3_indicateurs/traces_collaboration_Anis_dispersion/tmp/collab_Anis_dispersion.csv')['indice_collab']

co_ecriture_sans_ponderation = pd.read_csv(
    "/Users/anis/test_labnbook/test_3_indicateurs/traces_collaboration_Anis_sans_ponderation/tmp/collab_Anis_sans_ponderation.csv")["indice_collab"]


data.insert(7, 'co_ecriture', co_ecriture)
data.insert(8, 'co_ecriture_sans_ponderation', co_ecriture_sans_ponderation)

data.insert(9, 'co_ecriture_dispersion', co_ecriture_dispersion)

# Garder que la dérniére trace de chaque labdoc
idx_labdocs = data['id_labdoc'].diff(
)[data['id_labdoc'].diff() != 0].index.values
idx_labdocs = np.delete(idx_labdocs, 0) - 1
data = data[data.index.isin(idx_labdocs)]


data = data.drop(data[data['n_users'] == 1].index).reset_index()
data = data.drop([ "id_mission", "id_report", "id_trace","index"], axis=1)
display(data.info(),data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20295 entries, 0 to 20294
Data columns (total 7 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id_labdoc                     20295 non-null  int64  
 1   n_users                       20295 non-null  int64  
 2   n_mots                        20295 non-null  int64  
 3   equilibre_contribution        20295 non-null  float64
 4   co_ecriture                   20295 non-null  float64
 5   co_ecriture_sans_ponderation  20295 non-null  float64
 6   co_ecriture_dispersion        20295 non-null  float64
dtypes: float64(4), int64(3)
memory usage: 1.1 MB


None

,id_labdoc,n_users,n_mots,equilibre_contribution,co_ecriture,co_ecriture_sans_ponderation,co_ecriture_dispersion
0,313544,3,67,0.089902,0.086970,0.097949,0.175216
1,325259,2,180,0.947732,0.616471,0.606979,0.656825
2,313676,3,110,0.036662,0.035226,0.039054,0.073016
3,326302,3,180,0.000000,0.000000,0.000000,0.000000
4,313555,3,152,0.069033,0.066678,0.147966,0.261730


In [3]:
# Je récupère  l'id_action = 9  (trace de modification)  de tous les labdocs dans la table trace
trace = pd.read_sql(
    " SELECT id_labdoc, action_time from trace WHERE  id_action=9 Order By id_labdoc ASC, action_time ASC", conn)


In [4]:
# Je fusionne trace et data avec l'intersection en id_labdoc pour avoir uniquement la trace des labdocs dont j'ai besoin 
df2 = pd.merge(data, trace, how='inner', on=['id_labdoc'])
#display(df2.head(),df2.info())

In [5]:
# Je récupére le nombre de trace modify de chaque labdoc 
size = df2.groupby(df2.id_labdoc,  as_index=False).size()
#size.head()

In [6]:
# Je rajoute ce nombre a la table 
df2 = pd.merge(df2, size, how='inner', on=['id_labdoc'])
#df2.head()

In [7]:
# Récupérer les index du changement de labdoc 
idx_time_change = df2['id_labdoc'].diff()[df2['id_labdoc'].diff() != 0].index.values

In [8]:
#debut
debut = df2.iloc[idx_time_change[:-1]].reset_index()
debut.drop(["index"],axis=1,inplace=True )


In [9]:
#fin
fin_idx = np.delete(idx_time_change - 1, 0)
fin = df2.iloc[fin_idx].reset_index()
fin.drop(["index"], axis=1, inplace=True)


In [10]:
fin["time"] = fin.action_time - debut.action_time
df = fin.drop(["action_time"], axis = 1)
df.columns = ["id_labdoc"	,"n_users",	"n_mots", 'equilibre_contribution', 'co_ecriture', 'co_ecriture_sans_ponderation', 'co_ecriture_dispersion', "n_modif", "duree"]
#df 

In [11]:
df.duree = df.duree.dt.total_seconds()
#df.info()

In [12]:
# Je recupére encore une fois la table trace 
trace2 = pd.read_sql(
    " SELECT id_labdoc, id_user from trace WHERE  id_action=9 Order By id_labdoc ASC, action_time ASC", conn)


In [13]:
# Je merge la table des traces dans la bdd avec mon data frame pour garder que les labdocs qui m'intéressent 
trace_df_merge = pd.merge(df, trace2, how='inner', on=['id_labdoc'])

In [14]:
# Je calcule l'indice de coécriture 
indic_coec = []
for _, id in enumerate(trace_df_merge["id_labdoc"].unique()):
    indic_coec.append(
        np.sum(trace_df_merge[trace_df_merge["id_labdoc"] == id]["id_user"].diff() != 0)-1)

df["indice_coec"] = indic_coec
df.rename(columns={'indice_coec': 'passage_main'}, inplace=True)
df.rename(columns={'duree': 'plage_travail'}, inplace=True)
df.rename(columns={'n_modif': 'durée_écriture'}, inplace=True)
df.durée_écriture = df.durée_écriture *20

In [15]:
# J'ai remarquer une anomalie sur certains labdoc. Il on un score de coécriture 0 et des indice de collaboration contribution non nulles. Je supprime ces labdocs 
df.drop(df[df["equilibre_contribution"] == 0].index, axis=0, inplace=True)
df.drop(df[df["passage_main"] == 0].index, axis=0, inplace=True)
# Je supprime un labdocs avec des score pour tous les indicateurs qui est négative (je sais pas prq)
df.drop(index=df[df["equilibre_contribution"]
               < 0].index.values, inplace=True)
# Enléver les labdocs ayant un une plage de travail supérieur au temps d'écriture
df = df[df.plage_travail > df.durée_écriture]
# Enlever les labdocs ayant un nombre de mots < 15
df = df[df["n_mots"] > 15]
columns_filtred = ["n_mots", "plage_travail", "durée_écriture"]

for x in columns_filtred:
    q75, q25, intr_qr, max, min = 0, 0, 0, 0, 0
    q75, q25 = np.percentile(sorted(df.loc[:, x]), [
                             75, 25], interpolation='midpoint')
    intr_qr = q75-q25

    max = q75+(1.5*intr_qr)
    min = q25-(1.5*intr_qr)

    df.loc[df[x] <= min, x] = np.nan
    df.loc[df[x] >= max, x] = np.nan
df.dropna(axis=0, inplace=True)


In [16]:
df.isna().any()


id_labdoc                       False
n_users                         False
n_mots                          False
equilibre_contribution          False
co_ecriture                     False
co_ecriture_sans_ponderation    False
co_ecriture_dispersion          False
durée_écriture                  False
plage_travail                   False
passage_main                    False
dtype: bool

In [17]:
df.to_excel(
    "/Users/anis/test_labnbook/test_3_indicateurs/comparaison/ADD_Labnbook_new_indicators/data.xlsx")
df.to_csv(
    "/Users/anis/test_labnbook/test_3_indicateurs/comparaison/ADD_Labnbook_new_indicators/data.csv")
